# Description

- Initialize with returns, predictions, target volatility, and oos start date
- Evaluate portfolios generated from the predictions

- TODO(gp): This should be called `Master_model_evaluator` like the class

# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import logging

import core.config as cconfig
import core.dataflow_model.model_evaluator as modeval
import core.dataflow_model.model_plotter as modplot
import helpers.dbg as dbg
import helpers.printing as hprint

In [ ]:
dbg.init_logger(verbosity=logging.INFO)
# dbg.init_logger(verbosity=logging.DEBUG)

_LOG = logging.getLogger(__name__)

# _LOG.info("%s", env.get_system_signature()[0])

hprint.config_notebook()

In [ ]:
# file_name = "/app/oos_experiment.RH2Eg.v2_0-top10.5T.run1_test/result_1/result_bundle.v2_0.pkl"

# import helpers.pickle_ as hpickle

# obj = hpickle.from_pickle(file_name)

In [ ]:
#obj.keys()

In [ ]:
# import helpers.introspection as hintro

# for k in obj.keys():
#     print(k, hintro.get_size_in_bytes(obj[k]))

In [ ]:
#obj["payload"]["fit_result_bundle"]["result_df"]

# Notebook config

In [ ]:
# Read from env var.
eval_config = cconfig.Config.from_env_var("AM_CONFIG_CODE")

# Override config.
if eval_config is None:
    # experiment_dir = "s3://eglp-spm-sasm/experiments/experiment.RH2Ef.v1_9-all.5T.20210831-004747.run1.tgz"
    experiment_dir = "/cache/experiments/oos_experiment.RH1E.v2_0-top100.5T"
    #experiment_dir = "/app/rc_experiment.RH8Ec.v2_0-top2.5T.2009.run1"
    aws_profile = None
    selected_idxs = None

    eval_config = cconfig.get_config_from_nested_dict(
        {
            "load_experiment_kwargs": {
                "src_dir": experiment_dir,
                "file_name": "result_bundle.v2_0.pkl",
                "experiment_type": "ins_oos",
                "selected_idxs": None,
                "aws_profile": aws_profile,
            },
            "model_evaluator_kwargs": {
                "predictions_col": "mid_ret_0_vol_adj_clipped_2_hat",
                "target_col": "mid_ret_0_vol_adj_clipped_2",
                # "oos_start": "2017-01-01",
                "oos_start": None,
                "abort_on_error": True,
            },
            "bh_adj_threshold": 0.1,
            "resample_rule": "W",
            "mode": "ins",
            "target_volatility": 0.1,
        }
    )
   
print(str(eval_config))

# Initialize ModelEvaluator and ModelPlotter

In [ ]:
# Build the ModelEvaluator from the eval config.
evaluator = modeval.ModelEvaluator.from_eval_config(eval_config)

# Build the ModelPlotter.
plotter = modplot.ModelPlotter(evaluator)

# Initialize ModelEvaluator and ModelPlotter

In [ ]:
load_config = eval_config["load_experiment_kwargs"].to_dict()

# Load only the columns needed by the ModelEvaluator.
load_config["load_rb_kwargs"] = {
    "columns": [
        eval_config["model_evaluator_kwargs"]["target_col"],
        eval_config["model_evaluator_kwargs"]["predictions_col"],
    ]
}
result_bundle_dict = cdmu.load_experiment_artifacts(**load_config)

# Build the ModelEvaluator.
evaluator = modeval.ModelEvaluator.from_result_bundle_dict(
    result_bundle_dict,
    # abort_on_error=False,
    abort_on_error=True,
    **eval_config["model_evaluator_kwargs"].to_dict(),
)
# Build the ModelPlotter.
plotter = modplot.ModelPlotter(evaluator)

In [ ]:
for i in range(3):
    egid = result_bundle_dict[i].config["load_prices"]["source_node_kwargs"]["func_kwargs"]["egid"]
    print(egid)

In [ ]:
import vendors_lime.taq_bars.utils as vltbut
import datetime

import pandas as pd
import numpy as np

In [ ]:
# Load the 1min data.
egid = 10025
#egid = 10035
#egid = 10036
columns = ['end_time', 'close', 'volume', 'egid', 'good_ask', 'good_bid', 'good_bid_size', 'good_ask_size']
cache_dir = "/cache/vltbut.get_bar_data.v2_1-all.2009_2019.20210907-07_52_53/cache.get_bar_data.v2_0-all.2009_2019"
df_1min = vltbut.load_single_instrument_data(egid, datetime.date(2009, 1, 1), datetime.date(2019, 1, 1), columns=columns, cache_dir=cache_dir)

df_1min.head()

In [ ]:
columns = ["close", "good_bid", "good_ask"]
df_1min[columns].resample("1B").mean().plot()
df_1min[columns].pct_change().resample("1B").mean().plot()
thr = 0.002
df_1min_clean = vltbut.clean_bars(df_1min, columns, thr)

df_1min_clean[columns].resample("1B").mean().plot()
df_1min_clean[columns].pct_change().resample("1B").mean().plot()

In [ ]:
# thr = 0.001
# for column in columns:
#     lower_bound = df_1min[column].quantile(thr)
#     upper_bound = df_1min[column].quantile(1.0 - thr)
#     mask = (df_1min[column] <= lower_bound) | (df_1min[column] >= upper_bound)
#     #print(mask)
#     #df_1min.loc[mask].resample("1B").mean().plot()
#     df_1min[mask] = np.nan
# #df_1min[df_1min <= lower_bound] = np.nan

# #df_1min[["good_ask", "good_bid", "close"]].quantile(0.99)
# df_1min[["good_ask", "good_bid", "close"]].resample("1B").mean().plot()
# #df_1min[["close"]].resample("1B").mean().plot()

In [ ]:
# mask = df_1min["close"] < 0.40
# df_1min[mask]

# ret_0 = df_1min.pct_change()
# ret_0.resample("1B").mean().plot()

In [ ]:
# #mad_func = lambda x: np.fabs(x - x.mean()).mean()
# #mad = df_1min["close"].dropna().rolling(window=60).apply(mad_func, raw=True)

# mad = df_1min["close"].dropna().rolling(window=60).quantile(.01)

# #df_1min["close"] +
# #.plot(style='k')

# mad.resample("1B").mean().plot()

In [ ]:
# df_1min["close"].hist(bins=101)
# df_1min[["close"]].min()

In [ ]:
df_1min_out = df_1min_clean[columns].resample("1T", closed="right", label="right").mean()
#mask = df_1min_out < 0.40
#df_1min_out = df_1min_out[~mask]

df_1min_out.fillna(method="ffill", limit=None, inplace=True)
# .sum(min_count=1) #.replace(np.nan, 0)

df_1min_out.columns = ["price", "bid", "ask"]
df_1min_out.resample("1B").mean().plot()

df_1min_out.dropna().head()

In [ ]:
#df_1min_out["price"].hist(bins=101)

In [ ]:
df_price = df_1min_out.resample("5T", closed="right", label="right").last()
df_price["ret_0"] = df_price["price"].pct_change()
df_price.resample("1D").mean().plot()

In [ ]:
#import pandas as pd
#df_5mins.loc[pd.Timestamp("2009-01-02 17:00:00-05:00")]

In [ ]:
#df_1min_out.loc[pd.Timestamp("2009-01-05 13:40:00-05:00"):pd.Timestamp("2009-01-05 14:00:00-05:00")]

In [ ]:
df_1min_out["midpoint"] = (df_1min_out["bid"] + df_1min_out["ask"]) / 2

In [ ]:
df_1min_out.head()

In [ ]:
import core.dataflow_model.pnl_simulator as pnlsim

df_5mins = result_bundle_dict[0].result_df[["mid_ret_0_vol_adj_clipped_2_hat"]]
df_5mins.columns = ["preds"]
df_5mins.dropna(inplace=True)


initial_wealth = 1e6
config = {
    "price_column": "price",
    "future_snoop_allocation": False,
    "order_type": "price@end",
    #"order_type": "midpoint@end",
    #"order_type": "full_spread@end",
    "use_cache": True,
    "cached_columns": ["price", "midpoint", "bid", "ask"]
}
mi = pnlsim.MarketInterface(df_1min_out, config["use_cache"], columns=config.get("cached_columns", None))

In [ ]:
df_5mins_out = pnlsim.compute_pnl_level2(mi, df_5mins, initial_wealth, config)
#wealth, ret, df_5mins_out = pnlsim.compute_pnl_level1(initial_wealth, df_1min_out, df_5mins)

df_5mins_out["wealth"].resample("1B").mean().plot()#["2012-01-01":].plot()

In [ ]:
config ["order_type"] = "midpoint@end"
df_5mins_out = pnlsim.compute_pnl_level2(mi, df_5mins, initial_wealth, config)
#wealth, ret, df_5mins_out = pnlsim.compute_pnl_level1(initial_wealth, df_1min_out, df_5mins)

df_5mins_out["wealth"].resample("1B").mean().plot()#["2012-01-01":].plot()

In [ ]:
config ["order_type"] = "midpoint@twap"
df_5mins_out = pnlsim.compute_pnl_level2(mi, df_5mins, initial_wealth, config)

df_5mins_out["wealth"].resample("1B").mean().plot()#["2012-01-01":].plot()


In [ ]:
config ["order_type"] = "partial_spread_0.5@end"
df_5mins_out = pnlsim.compute_pnl_level2(mi, df_5mins, initial_wealth, config)
#wealth, ret, df_5mins_out = pnlsim.compute_pnl_level1(initial_wealth, df_1min_out, df_5mins)

df_5mins_out["wealth"].resample("1B").mean().plot()#["2012-01-01":].plot()

In [ ]:
config ["order_type"] = "partial_spread_0.0@end"
df_5mins_out = pnlsim.compute_pnl_level2(mi, df_5mins, initial_wealth, config)
#wealth, ret, df_5mins_out = pnlsim.compute_pnl_level1(initial_wealth, df_1min_out, df_5mins)

df_5mins_out["wealth"].resample("1b").mean().plot()#["2012-01-01":].plot()

In [ ]:
config ["order_type"] = "partial_spread_0.3@end"
df_5mins_out = pnlsim.compute_pnl_level2(mi, df_5mins, initial_wealth, config)
df_5mins_out["wealth"].resample("1b").mean().plot()#["2012-01-01":].plot()

In [ ]:
config ["order_type"] = "partial_spread_0.51@end"
df_5mins_out = pnlsim.compute_pnl_level2(mi, df_5mins, initial_wealth, config)
df_5mins_out["wealth"].resample("1b").mean().plot()#["2012-01-01":].plot()

In [ ]:
config ["order_type"] = "full_spread@end"
df_5mins_out = pnlsim.compute_pnl_level2(mi, df_5mins, initial_wealth, config)
#wealth, ret, df_5mins_out = pnlsim.compute_pnl_level1(initial_wealth, df_1min_out, df_5mins)

df_5mins_out["wealth"].resample("1B").mean().plot()#["2012-01-01":].plot()

In [ ]:
df_all2 = pd.DataFrame()
for order_type in [
                   "partial_spread_0.5@end", "midpoint@end", "midpoint@twap"
]:
    config ["order_type"] = order_type
    df_5mins_out = pnlsim.compute_pnl_level2(mi, df_5mins, initial_wealth, config)
    df_all2[order_type] = df_5mins_out["wealth"]

df_all2.resample("1B").mean().plot()#["2012-01-01":].plot()

In [ ]:
df_all = pd.DataFrame()
for order_type in [
                   #"partial_spread_0.3@end", 
                   #"partial_spread_0.4@end",
                   "partial_spread_0.45@end", "partial_spread_0.5@end",
                  "partial_spread_0.51@end",
                  "partial_spread_0.52@end",
                  "partial_spread_0.53@end"]: 
    config ["order_type"] = order_type
    df_5mins_out = pnlsim.compute_pnl_level2(mi, df_5mins, initial_wealth, config)
    df_all[order_type] = df_5mins_out["wealth"]

df_all.resample("1B").mean().plot()#["2012-01-01":].plot()

In [ ]:
df_5mins_out.tail()

In [ ]:
df_5mins_out["wealth"].resample("1B").mean().plot()#["2012-01-01":].plot()

In [ ]:
df_5mins_out["wealth"].resample("1B").mean().plot()#["2012-01-01":].plot()

In [ ]:
df_5mins_merged = df_5mins_out.merge(df_price, right_index=True, left_index=True)

_, df_5mins_merged = pnlsim.compute_lag_pnl(df_5mins_merged)
#display(df_5mins_merged)
#df_5mins_merged["pnl.lag"].cumsum().plot()
df_5mins_merged["pnl.sim1"].cumsum().plot()

In [ ]:
# Build the ModelEvaluator from the eval config.
evaluator = modeval.ModelEvaluator.from_eval_config(eval_config)

# Build the ModelPlotter.
plotter = modplot.ModelPlotter(evaluator)

# Analysis

In [ ]:
pnl_stats = evaluator.calculate_stats(
    mode=eval_config["mode"], target_volatility=eval_config["target_volatility"]
)
display(pnl_stats)

## Model selection

In [ ]:
plotter.plot_multiple_tests_adjustment(
    threshold=eval_config["bh_adj_threshold"], mode=eval_config["mode"]
)

In [ ]:
# TODO(gp): Move this chunk of code in a function.
col_mask = (
    pnl_stats.loc["signal_quality"].loc["sr.adj_pval"]
    < eval_config["bh_adj_threshold"]
)
selected = pnl_stats.loc[:, col_mask].columns.to_list()
not_selected = pnl_stats.loc[:, ~col_mask].columns.to_list()

print("num model selected=%s" % hprint.perc(len(selected), pnl_stats.shape[1]))
print("model selected=%s" % selected)
print("model not selected=%s" % not_selected)

# Use `selected = None` to show all the models.

In [ ]:
# selected = None
plotter.plot_multiple_pnls(
    keys=selected,
    resample_rule=eval_config["resample_rule"],
    mode=eval_config["mode"],
)

## Return correlation

In [ ]:
plotter.plot_correlation_matrix(
    series="returns",
    resample_rule=eval_config["resample_rule"],
    mode=eval_config["mode"],
)

In [ ]:
plotter.plot_effective_correlation_rank(
    series="returns",
    resample_rule=eval_config["resample_rule"],
    mode=eval_config["mode"],
)

## Model correlation

In [ ]:
plotter.plot_correlation_matrix(
    series="pnl",
    resample_rule=eval_config["resample_rule"],
    mode=eval_config["mode"],
)

In [ ]:
plotter.plot_effective_correlation_rank(
    series="pnl",
    resample_rule=eval_config["resample_rule"],
    mode=eval_config["mode"],
)

## Aggregate model

In [ ]:
pnl_srs, pos_srs, aggregate_stats = evaluator.aggregate_models(
    keys=selected,
    mode=eval_config["mode"],
    target_volatility=eval_config["target_volatility"],
)
display(aggregate_stats)

In [ ]:
plotter.plot_sharpe_ratio_panel(keys=selected, mode=eval_config["mode"])

In [ ]:
plotter.plot_rets_signal_analysis(
    keys=selected,
    resample_rule=eval_config["resample_rule"],
    mode=eval_config["mode"],
    target_volatility=eval_config["target_volatility"],
)

In [ ]:
plotter.plot_performance(
    keys=selected,
    resample_rule=eval_config["resample_rule"],
    mode=eval_config["mode"],
    target_volatility=eval_config["target_volatility"],
)

In [ ]:
plotter.plot_rets_and_vol(
    keys=selected,
    resample_rule=eval_config["resample_rule"],
    mode=eval_config["mode"],
    target_volatility=eval_config["target_volatility"],
)

In [ ]:
if False:
    plotter.plot_positions(
        keys=selected,
        mode=eval_config["mode"],
        target_volatility=eval_config["target_volatility"],
    )

In [ ]:
if False:
    # Plot the returns and prediction for one or more models.
    model_key = selected[:1]
    plotter.plot_returns_and_predictions(
        keys=model_key,
        resample_rule=eval_config["resample_rule"],
        mode=eval_config["mode"],
    )